<a href="https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/semi-supervised/Noisy_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semi-surpervise model

當我們蒐集到大量的資料，一部分有標籤，一部分沒有標籤，有標籤的資料不足以讓我們建構一個神經網路時 (標籤資料太少)，這種情況，就稱為半監督學習

半監督學習有很多種方法，我們今天來聊聊其中三種 Noisy Student、$\pi$-Model、Temporal Ensembling

## [Noisy Student](https://arxiv.org/abs/1911.04252)

如果讓我用簡單的一句話介紹 NS，那麼我會說"在建立 pseudo label 的過程加上 augmentation"

### Pseudo Labeling 

偽標籤是機器學習常用的方法。當我們訓練樣本不夠多時，我們可以先用少量的訓練樣本建立一個模型，再把測試集的資料輸入模型生成標籤，因為標籤是模型生成的，並不是真的，所以稱為偽標籤，之後把測試集的資料配上訓練集的資料一同訓練，這個過程就是 Pseudo Labeling

### Teacher and Student

同 Pseudo Labeling，我們先把資料分為兩類

- Labeled Data: 手動標籤數據
- Unlabeled Data: 無標籤數據

Note: Labeled Data 通常遠小於 Unlabeled Data

我們會先用 Labeled Data 訓練出一個 baseline 模型，這個模型稱為 Teacher model，
之後將無標籤的資料進行各種不同的數據增強，或者 Stochastic depth、dropout 等方法輸入 Teacher model 得到偽標籤，之後放入另一個模型訓練，這個模型稱為 Student model。當學生模型訓練成功後，我們會把學生模型當成教師模型，再訓練下一個學生模型，直到收斂

![](https://miro.medium.com/max/863/1*w7MBsZs0xtQrGtR24-AXHg.png)

整個流程可以簡化為:

1. 使用 Labeled Data 訓練 Teacher Model
2. 利用 Teacher Model 預測偽標籤 (can be soft label or hard label)
3. 將兩筆資料合併並且加上 noise 放入 Student Model 訓練
4. 用 Student Model 代替舊的 Teacher Model 成為新的 Teacher Model

但在過程中需要注意

1. Student Model 需要至多大於 Teacher Model (需要適應更大的資料集)
2. Balance Data 對於 Student Model 有顯著提升

## $\pi$-Model and Temporal Ensembling

![](https://www.researchgate.net/publication/350087961/figure/fig2/AS:1001824706371584@1615865031042/Difference-between-P-model-and-Temporal-ensembling-Laine-and-Aila-2016.ppm)

$\pi$-Model 的流程如下:

1. 將輸入做兩種數據增強方法
2. 通過網路後得到輸出 $z_i$ 和 $\tilde z_i$，並計算平方差 $||z_i-\tilde z_i||^2$
3. 若數據為 Labeled Data，計算 $y_i$ 與 $z_i$ 的交叉墒
4. 將兩者做加權平均 $w(t)$

注意到，若一開始的數據皆為 Unlabeled Data，則模型只透過平方差學不到什麼東西，所以我們會先用 Labeled Data 訓練，並且在加權項 $w(t)$ 隨著時間推移將用來用偏重 unsurprised loss

![](https://images2015.cnblogs.com/blog/798706/201703/798706-20170328144155436-204997815.png)

其實 Temporal ensemble model 與 $\pi-$Model 大致上很像，唯一的不同在於 $\tilde z$ 的選擇上，Temporal ensemble model 中的 $\tilde z$ 是過去歷史 $z$ 的加權和，隨著時間推移，越早的 $\tilde z$ 將會越來越不重要

![](https://images2015.cnblogs.com/blog/798706/201703/798706-20170328151230014-1510280027.png)